# 🚀 Orchestration Layer - End-to-End Testing

This notebook tests the **complete RAG pipeline** with all agents coordinated.

## What is Tested

- ✅ Router Agent classification
- ✅ Needle Agent execution (precise facts)
- ✅ Summary Agent execution (contextual synthesis)
- ✅ End-to-end pipeline flow
- ✅ Unified response format
- ✅ Correct agent selection

## Test Strategy

**End-to-End Approach:**
- Uses mock retrievers for self-contained testing
- Tests complete pipeline: Question → Route → Agent → Answer
- Verifies correct agent is chosen for each question type
- Validates unified output schema


---

## 📦 Setup


In [1]:
# Imports
import os
import sys
from pathlib import Path
from typing import List
from dataclasses import dataclass

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")


✅ Project root: /Users/guyai/Desktop/AI Lecture/FIRST PROJECT/RagAgentv2


In [2]:
# Load environment
from dotenv import load_dotenv

env_path = project_root / ".env"
load_dotenv(env_path)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

print("✅ Environment loaded")
print(f"   OpenAI API Key: {'*' * 20}{api_key[-4:]}")


✅ Environment loaded
   OpenAI API Key: ********************NQQA


---

## 🔧 Mock Retrievers

We'll reuse the mock retrievers from individual agent tests.


In [3]:
# Mock objects to simulate LlamaIndex retriever output

@dataclass
class MockNode:
    """Mock LlamaIndex Node."""
    node_id: str
    text: str

@dataclass
class MockNodeWithScore:
    """Mock LlamaIndex NodeWithScore."""
    node: MockNode
    score: float


# ============================================================
# MOCK NEEDLE RETRIEVER (for atomic facts)
# ============================================================

class MockNeedleRetriever:
    """Mock retriever for needle questions - returns atomic chunks."""
    
    def __init__(self):
        self.knowledge_base = {
            "claimant_name": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_001",
                    text="Claimant Information: Name: John Michael Doe, Age: 35 years old."
                ),
                score=0.95
            ),
            "claimant_phone": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_002",
                    text="Contact Details: Phone: 555-123-4567, Address: 123 Main Street."
                ),
                score=0.92
            ),
            "accident_date": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_003",
                    text="Incident occurred on January 15, 2024 at approximately 2:30 PM."
                ),
                score=0.94
            ),
            "claim_number": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_004",
                    text="Claim Reference Number: CLM-2024-00789. Policy Number: POL-556789."
                ),
                score=0.96
            ),
            "vin": MockNodeWithScore(
                node=MockNode(
                    node_id="chunk_005",
                    text="Vehicle: 2020 Honda Accord, VIN: 1HGCV1F30LA123456, Color: Silver."
                ),
                score=0.93
            ),
        }
    
    def retrieve(self, question: str) -> List[MockNodeWithScore]:
        """Mock needle retrieval."""
        question_lower = question.lower()
        results = []
        
        if "name" in question_lower and "claimant" in question_lower:
            results.append(self.knowledge_base["claimant_name"])
        if "phone" in question_lower:
            results.append(self.knowledge_base["claimant_phone"])
        if "when" in question_lower or "date" in question_lower:
            results.append(self.knowledge_base["accident_date"])
        if "claim number" in question_lower:
            results.append(self.knowledge_base["claim_number"])
        if "vin" in question_lower:
            results.append(self.knowledge_base["vin"])
        
        return results


# ============================================================
# MOCK SUMMARY RETRIEVER (for contextual synthesis)
# ============================================================

class MockSummaryRetriever:
    """Mock retriever for summary questions - returns merged parent chunks."""
    
    def __init__(self):
        self.knowledge_base = {
            "incident_overview": MockNodeWithScore(
                node=MockNode(
                    node_id="parent_001",
                    text="""INCIDENT OVERVIEW:
The accident occurred on January 15, 2024 at approximately 2:30 PM on Interstate 95, 
northbound lane near Exit 42. Weather conditions at the time included heavy rain with 
poor visibility (estimated 100 feet). Traffic was moving slowly due to weather.

The incident involved two vehicles: a 2020 Honda Accord (claimant vehicle) and a 
2019 Toyota Camry (other party). The Honda Accord was traveling in the middle lane 
when traffic suddenly slowed. The vehicle attempted to brake but was unable to stop 
in time, resulting in a rear-end collision with the Toyota Camry ahead.

No injuries were reported at the scene. Both drivers exchanged information and 
remained at the location until police arrived."""
                ),
                score=0.92
            ),
            "damage_assessment": MockNodeWithScore(
                node=MockNode(
                    node_id="parent_002",
                    text="""DAMAGE ASSESSMENT AND REPAIRS:
Initial damage estimate: $4,500

Damage description:
- Front bumper: Significant damage, requires replacement
- Hood: Minor dent, paintless dent repair recommended
- Front grille: Cracked, requires replacement
- Right headlight: Housing cracked, full replacement needed
- Radiator: No visible damage, but inspection recommended

Adjuster assigned: Sarah Johnson, assigned January 16, 2024
Inspection completed: January 18, 2024
Repair shop: AutoFix Collision Center, 456 Repair Ave"""
                ),
                score=0.91
            ),
            "claim_timeline": MockNodeWithScore(
                node=MockNode(
                    node_id="parent_003",
                    text="""CLAIM PROCESSING TIMELINE:
Claim Number: CLM-2024-00789
Filed: January 15, 2024, 6:45 PM (same day as incident)
Status: Approved

Processing timeline:
- January 15: Claim filed online
- January 16: Adjuster Sarah Johnson assigned
- January 18: Vehicle inspection completed
- January 18: Damage estimate approved
- January 19: Repair authorization sent
- January 22: Repairs scheduled to begin"""
                ),
                score=0.90
            ),
        }
    
    def retrieve(self, question: str) -> List[MockNodeWithScore]:
        """Mock summary retrieval."""
        question_lower = question.lower()
        results = []
        
        if any(word in question_lower for word in ["accident", "incident", "happen", "describe"]):
            results.append(self.knowledge_base["incident_overview"])
        
        if any(word in question_lower for word in ["damage", "repair", "broken"]):
            results.append(self.knowledge_base["damage_assessment"])
        
        if any(word in question_lower for word in ["timeline", "process", "filed"]):
            results.append(self.knowledge_base["claim_timeline"])
        
        # Broad questions get multiple chunks
        if any(word in question_lower for word in ["summary", "overview", "everything"]):
            results = list(self.knowledge_base.values())
        
        return results


# Create mock retrievers
needle_retriever = MockNeedleRetriever()
summary_retriever = MockSummaryRetriever()

print("✅ Mock retrievers created")
print(f"   Needle retriever: {len(needle_retriever.knowledge_base)} atomic chunks")
print(f"   Summary retriever: {len(summary_retriever.knowledge_base)} merged chunks")


✅ Mock retrievers created
   Needle retriever: 5 atomic chunks
   Summary retriever: 3 merged chunks


---

## 🤖 Initialize All Agents


In [4]:
from RAG.Agents import RouterAgent, NeedleAgent, SummaryAgent

# Initialize all agents
print("Initializing agents...")
print("="*70)

router_agent = RouterAgent(model="gpt-4o-mini", temperature=0.0)
print()

needle_agent = NeedleAgent(model="gpt-4o-mini", temperature=0.0)
print()

summary_agent = SummaryAgent(model="gpt-4o-mini", temperature=0.2)
print()

print("="*70)
print("✅ All agents initialized successfully")


Initializing agents...
✅ Router Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.0
   Output: Structured (Pydantic)

✅ Needle Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.0
   Output: Structured (Pydantic)
   Policy: NO GUESSING

✅ Summary Agent initialized
   Model: gpt-4o-mini
   Temperature: 0.2
   Output: Structured (Pydantic)
   Policy: CONTEXT-GROUNDED SYNTHESIS

✅ All agents initialized successfully


---

## 🎯 Create Orchestrator


In [5]:
from RAG.Orchestration import Orchestrator

# Create orchestrator with all dependencies
orchestrator = Orchestrator(
    router_agent=router_agent,
    needle_agent=needle_agent,
    summary_agent=summary_agent,
    needle_retriever=needle_retriever,
    summary_retriever=summary_retriever
)

print("\n" + "="*70)
print("🚀 Orchestrator ready for end-to-end testing!")
print("="*70)


✅ Orchestrator initialized
   Router Agent: RouterAgent
   Needle Agent: NeedleAgent
   Summary Agent: SummaryAgent
   Needle Retriever: MockNeedleRetriever
   Summary Retriever: MockSummaryRetriever

🚀 Orchestrator ready for end-to-end testing!


---

## 🧪 Test Suite - End-to-End

We'll test the complete pipeline with both needle and summary questions.


In [6]:
# Helper function for testing
def test_pipeline(question: str, expected_route: str):
    """
    Test the complete RAG pipeline.
    
    Args:
        question: Question to test
        expected_route: Expected routing decision ("needle" or "summary")
    """
    print("\n" + "█"*70)
    print(f"TEST QUESTION: {question}")
    print("█"*70)
    
    # Run pipeline
    result = orchestrator.run(question)
    
    # Validate routing
    actual_route = result["route"]
    if actual_route == expected_route:
        print(f"\n✅ ROUTING CORRECT: Expected {expected_route}, got {actual_route}")
    else:
        print(f"\n❌ ROUTING INCORRECT: Expected {expected_route}, got {actual_route}")
    
    # Validate response schema
    required_keys = ["route", "answer", "confidence", "sources", "reason"]
    missing_keys = [k for k in required_keys if k not in result]
    
    if not missing_keys:
        print("✅ RESPONSE SCHEMA: All required fields present")
    else:
        print(f"❌ RESPONSE SCHEMA: Missing fields: {missing_keys}")
    
    return result


### Test 1: Needle Question - Claimant Name


In [7]:
test_pipeline(
    question="What is the claimant's name?",
    expected_route="needle"
)



██████████████████████████████████████████████████████████████████████
TEST QUESTION: What is the claimant's name?
██████████████████████████████████████████████████████████████████████

🚀 RAG PIPELINE STARTED
Question: What is the claimant's name?

[STEP 1] ROUTING
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Confidence: 1.00
✓ Reason:     The question asks for a specific piece of information, which is the claimant's name, making it a clear case for the NEEDLE route.

[STEP 2] EXECUTION
──────────────────────────────────────────────────────────────────────
→ Executing NEEDLE AGENT...

🔍 Retrieving chunks for: 'What is the claimant's name?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

[STEP 3] RESPONSE
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Answer:     John Michael Doe
✓ Confidence: 0.95
✓ Sources:    1 chunk(s)
✓ Reason:     Found explicitly in chunk_001

✅ R

{'route': 'needle',
 'answer': 'John Michael Doe',
 'confidence': 0.95,
 'sources': ['chunk_001'],
 'reason': 'Found explicitly in chunk_001'}

### Test 2: Needle Question - Phone Number


In [8]:
test_pipeline(
    question="What is the claimant's phone number?",
    expected_route="needle"
)



██████████████████████████████████████████████████████████████████████
TEST QUESTION: What is the claimant's phone number?
██████████████████████████████████████████████████████████████████████

🚀 RAG PIPELINE STARTED
Question: What is the claimant's phone number?

[STEP 1] ROUTING
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Confidence: 1.00
✓ Reason:     The question asks for a specific piece of information, which is the claimant's phone number, making it a clear case for the NEEDLE route.

[STEP 2] EXECUTION
──────────────────────────────────────────────────────────────────────
→ Executing NEEDLE AGENT...

🔍 Retrieving chunks for: 'What is the claimant's phone number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

[STEP 3] RESPONSE
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Answer:     555-123-4567
✓ Confidence: 0.95
✓ Sources:    1 chunk(s)
✓ Reason:     Found 

{'route': 'needle',
 'answer': '555-123-4567',
 'confidence': 0.95,
 'sources': ['chunk_002'],
 'reason': 'Found explicitly in chunk_002'}

### Test 3: Needle Question - Claim Number


In [9]:
test_pipeline(
    question="What is the claim number?",
    expected_route="needle"
)



██████████████████████████████████████████████████████████████████████
TEST QUESTION: What is the claim number?
██████████████████████████████████████████████████████████████████████

🚀 RAG PIPELINE STARTED
Question: What is the claim number?

[STEP 1] ROUTING
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Confidence: 1.00
✓ Reason:     The question asks for a specific piece of information, which is the claim number, making it a clear case for the NEEDLE route.

[STEP 2] EXECUTION
──────────────────────────────────────────────────────────────────────
→ Executing NEEDLE AGENT...

🔍 Retrieving chunks for: 'What is the claim number?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

[STEP 3] RESPONSE
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Answer:     CLM-2024-00789
✓ Confidence: 0.95
✓ Sources:    1 chunk(s)
✓ Reason:     Found explicitly in chunk_004

✅ RAG PIPELINE CO

{'route': 'needle',
 'answer': 'CLM-2024-00789',
 'confidence': 0.95,
 'sources': ['chunk_004'],
 'reason': 'Found explicitly in chunk_004'}

### Test 4: Summary Question - Incident Description


In [ ]:
test_pipeline(
    question="Describe what happened in the accident.",
    expected_route="summary"
)


### Test 5: Summary Question - Damage Explanation


In [ ]:
test_pipeline(
    question="Explain the damage to the vehicle.",
    expected_route="summary"
)


### Test 6: Summary Question - Claim Timeline


In [ ]:
test_pipeline(
    question="Tell me about the claim processing timeline.",
    expected_route="summary"
)


---

## 📊 Batch Testing - Full Pipeline Validation


In [10]:
# Comprehensive test suite
test_cases = [
    # NEEDLE QUESTIONS
    ("What is the claimant's name?", "needle"),
    ("What is the claimant's phone number?", "needle"),
    ("When did the accident occur?", "needle"),
    ("What is the claim number?", "needle"),
    ("What is the VIN number?", "needle"),
    
    # SUMMARY QUESTIONS
    ("Describe what happened in the accident.", "summary"),
    ("Explain the damage to the vehicle.", "summary"),
    ("Tell me about the claim processing timeline.", "summary"),
    ("Summarize the entire incident.", "summary"),
]

print("\n" + "="*70)
print("🧪 COMPREHENSIVE PIPELINE TEST")
print("="*70)

passed = 0
failed = 0
results = []

for question, expected_route in test_cases:
    try:
        result = orchestrator.run(question)
        actual_route = result["route"]
        
        # Check routing correctness
        routing_correct = (actual_route == expected_route)
        
        # Check response schema
        required_keys = ["route", "answer", "confidence", "sources", "reason"]
        schema_valid = all(k in result for k in required_keys)
        
        # Check answer present
        has_answer = result["answer"] is not None and len(str(result["answer"])) > 0
        
        # Overall pass/fail
        test_passed = routing_correct and schema_valid and has_answer
        
        if test_passed:
            passed += 1
            status = "✅ PASS"
        else:
            failed += 1
            status = "❌ FAIL"
        
        results.append({
            "question": question,
            "expected_route": expected_route,
            "actual_route": actual_route,
            "routing_correct": routing_correct,
            "schema_valid": schema_valid,
            "has_answer": has_answer,
            "status": status
        })
        
    except Exception as e:
        failed += 1
        results.append({
            "question": question,
            "expected_route": expected_route,
            "error": str(e),
            "status": "❌ ERROR"
        })

print("\n" + "="*70)
print("📊 TEST RESULTS SUMMARY")
print("="*70)

for r in results:
    print(f"\n{r['status']}")
    print(f"  Q: {r['question']}")
    if 'error' in r:
        print(f"  Error: {r['error']}")
    else:
        print(f"  Expected route: {r['expected_route']}")
        print(f"  Actual route: {r['actual_route']}")
        print(f"  Routing: {'✓' if r['routing_correct'] else '✗'}")
        print(f"  Schema: {'✓' if r['schema_valid'] else '✗'}")
        print(f"  Answer: {'✓' if r['has_answer'] else '✗'}")

print("\n" + "="*70)
print(f"TOTAL: {passed} passed, {failed} failed out of {len(test_cases)} tests")
success_rate = (passed / len(test_cases)) * 100
print(f"SUCCESS RATE: {success_rate:.1f}%")
print("="*70)



🧪 COMPREHENSIVE PIPELINE TEST

🚀 RAG PIPELINE STARTED
Question: What is the claimant's name?

[STEP 1] ROUTING
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Confidence: 1.00
✓ Reason:     The question asks for a specific piece of information, which is the claimant's name, making it a clear case for the NEEDLE route.

[STEP 2] EXECUTION
──────────────────────────────────────────────────────────────────────
→ Executing NEEDLE AGENT...

🔍 Retrieving chunks for: 'What is the claimant's name?'
   ✅ Retrieved 1 chunk(s)
   🤖 Extracting fact with gpt-4o-mini...

[STEP 3] RESPONSE
──────────────────────────────────────────────────────────────────────
✓ Route:      NEEDLE
✓ Answer:     John Michael Doe
✓ Confidence: 0.95
✓ Sources:    1 chunk(s)
✓ Reason:     Found explicitly in chunk_001

✅ RAG PIPELINE COMPLETED

🚀 RAG PIPELINE STARTED
Question: What is the claimant's phone number?

[STEP 1] ROUTING
─────────────────────────────────────────────

---

## ✅ Conclusion

The **Orchestration Layer** successfully coordinates all agents:

### What Was Tested:

✅ **Router Agent** correctly classifies questions  
✅ **Needle Agent** executes for atomic fact questions  
✅ **Summary Agent** executes for contextual questions  
✅ **Unified response** format across all questions  
✅ **End-to-end pipeline** works seamlessly  

### Architecture Highlights:

🎯 **Single Entry Point** - `orchestrator.run(question)`  
🔀 **Explicit Routing** - Router Agent makes classification  
🤖 **Agent Delegation** - Appropriate agent handles question  
📦 **Dependency Injection** - All components injected  
🚫 **No Business Logic** - Pure coordination  

### Production Ready:

- ✅ Clean separation of concerns
- ✅ Easy to test (mock retrievers)
- ✅ Consistent output format
- ✅ No hidden dependencies
- ✅ Explicit error handling
- ✅ Stateless operation

**Your RAG system is now complete!** 🎉
